In [ ]:
import os
from itertools import chain
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import utils
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, GlobalAveragePooling2D, LSTM, TimeDistributed, Dropout, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator, array_to_img
import numpy as np
from PIL import Image
from tqdm import tqdm
import pandas as pd
import math
import matplotlib.pyplot as plt
import shutil

from data import Data

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
LABEL_INDEX = {
    'ap': 0,
    'bs': 1,
    'mid': 2,
    'oap': 3,
    'obs': 4,
}

In [ ]:
keras_app = tf.keras.applications.mobilenet
keras_model = tf.keras.applications.mobilenet.MobileNet
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1,
    fill_mode="nearest",
    preprocessing_function=keras_app.preprocess_input)
datagen = ImageDataGenerator(preprocessing_function=keras_app.preprocess_input)

In [ ]:
class PhaseDataGenerator(keras.utils.Sequence):
    def __init__(self, data: Data, datasets=None, batch_size=32, target_size=(224, 224), 
                 slices_per_sample=25, shuffle=True, image_data_generator=None):
        self.data = data
        self.datasets = datasets
        self.batch_size = batch_size
        self.target_size=  target_size
        self.slices_per_sample = slices_per_sample
        self.shuffle = shuffle
        self.datagen = image_data_generator
        
        self.n_classes = 5
        self.label_indices = LABEL_INDEX

        self.samples = dict()
        self.max_slices = 1
        
        if datasets is None:
            datasets = list(data.data.keys())
        if isinstance(datasets, str):
            datasets = [datasets]
        
        # All plural variables are dicts
        for dataset in datasets:
            for patient, phases in data.data[dataset].items():
                for phase, slices in phases.items():
                    key = "{dataset}_{patient:06d}_{slice:02d}".format(dataset=dataset, patient=patient, slice=phase)
                    self.samples[key] = slices
                    if len(slices) > self.max_slices:
                        self.max_slices = len(slices)
        
        if slices_per_sample < self.max_slices:
            raise ValueError("There are some samples that contain more than {} slices ({})".format(
                slices_per_sample, self.max_slices))
        
        unlabeled = []
        self.images_by_label = [0] * len(self.label_indices)
        for phases in self.samples.values():
            for phase in phases.values():
                label = self.data.labels.get(phase, None)
                if label is None:
                    unlabeled.append(phase)
                else:
                    index = self.label_indices[label]
                    self.images_by_label[index] += 1
        if unlabeled:
            raise ValueError("{} unlabeled slice(s): {}...".format(len(unlabeled), str(unlabeled)[:200]))
        
        self.n_batches = math.ceil(len(self.samples) / batch_size)
        
        self._refresh_sample_keys()
    
    def _refresh_sample_keys(self):
        self.sample_keys = sorted(list(self.samples.keys()))
        if self.shuffle:
            np.random.shuffle(self.sample_keys)
        
    def _get_sample_key_batch(self, index):
        return self.sample_keys[index * self.batch_size:(index + 1) * self.batch_size]

    def _load_and_preprocess_image(self, path, standardize=False):
        img = Image.open(path)
        img = img.resize(self.target_size, Image.NEAREST)
        #img = load_img(path, color_mode="rgb", target_size=self.target_size)
        x = img_to_array(img, data_format="channels_last")
        params = datagen.get_random_transform(x.shape)
        x = x / 65536 * 255
        x = datagen.apply_transform(x, params)
        if standardize:
            x = datagen.standardize(x)
        return x
    
    def get_class_weight(self):
        counts = np.array(self.images_by_label)
        weights = counts.sum() / counts / len(counts)
        weights = { i: weight for i, weight in enumerate(weights.tolist())}
        return weights
        
    def __getitem__(self, index):
        """Get `index`th batch
        """
        keys = self._get_sample_key_batch(index)
        batch_size = len(keys)
        x = np.zeros((batch_size, self.slices_per_sample) + self.target_size + (3,))
        y = np.zeros((batch_size, self.slices_per_sample) + (self.n_classes,))
        for i, key in enumerate(keys):
            items = sorted(list(self.samples[key].items()))
            for j, (slice_index, sid) in enumerate(items):
                path = self.data.paths[sid]
                image = self._load_and_preprocess_image(path, standardize=True)
                x[i][j] = image
                label = self.data.labels[sid]
                label_index = self.label_indices[label]
                y[i][j][label_index] = 1
        
        return x, y
    
    def __len__(self):
        return self.n_batches
    
    def on_epoch_end(self):
        self._refresh_sample_keys()

In [ ]:
class SliceDataGenerator(keras.utils.Sequence):
    def __init__(self, data: Data, datasets=None, batch_size=32,
                 target_size=(224, 224), shuffle=True, image_data_generator=None):
        self.data = data
        self.datasets = datasets
        self.batch_size = batch_size
        self.target_size=  target_size
        self.shuffle = shuffle
        self.datagen = image_data_generator
        
        self.n_classes = 5
        self.label_indices = LABEL_INDEX

        self.slices = list()

        if datasets is None:
            datasets = list(data.data.keys())
        if isinstance(datasets, str):
            datasets = [datasets]
        
        # All plural variables are dicts
        for dataset in datasets:
            for patient, phases in data.data[dataset].items():
                for phase, slices in phases.items():
                    ordered = sorted(list(slices.items()))
                    ordered_slices = [item[1] for item in ordered]
                    self.slices.extend(ordered_slices)
        self.slices.sort()

        unlabeled = []
        self.images_by_label = [0] * len(self.label_indices)
        for slice in self.slices:
            label = self.data.labels.get(slice, None)
            if label is None:
                unlabeled.append(slice)
            else:
                index = self.label_indices[label]
                self.images_by_label[index] += 1
        if unlabeled:
            raise ValueError("{} unlabeled slice(s): {}...".format(len(unlabeled), str(unlabeled)[:200]))
        
        self.n_batches = math.ceil(len(self.slices) / batch_size)
        self._refresh_slice_order()
    
    def _refresh_slice_order(self):
        self.slices = sorted(list(self.slices))
        if self.shuffle:
            np.random.shuffle(self.slices)
        
    def _get_slice_batch(self, index):
        return self.slices[index * self.batch_size:(index + 1) * self.batch_size]

    def _load_and_preprocess_image(self, path, standardize=False):
        img = Image.open(path)
        img = img.resize(self.target_size, Image.NEAREST)
        #img = load_img(path, color_mode="rgb", target_size=self.target_size)
        x = img_to_array(img, data_format="channels_last")
        params = datagen.get_random_transform(x.shape)
        x = x / 65536 * 255
        x = datagen.apply_transform(x, params)
        if standardize:
            x = datagen.standardize(x)
        return x
    
    def get_class_weight(self):
        counts = np.array(self.images_by_label)
        weights = counts.sum() / counts / len(counts)
        weights = { i: weight for i, weight in enumerate(weights.tolist())}
        return weights
        
    def __getitem__(self, index):
        """Get `index`th batch
        """
        slices = self._get_slice_batch(index)
        batch_size = len(slices)
        x = np.zeros((batch_size, ) + self.target_size + (3,))
        y = np.zeros((batch_size, ) + (self.n_classes,))
        for i, sid in enumerate(slices):
            path = self.data.paths[sid]
            image = self._load_and_preprocess_image(path, standardize=True)
            x[i] = image
            label = self.data.labels[sid]
            label_index = self.label_indices[label]
            y[i][label_index] = 1

        assert(y.sum() == len(slices))
        
        return x, y
    
    def __len__(self):
        return self.n_batches
    
    def on_epoch_end(self):
        self._refresh_slice_order()

In [ ]:
class SlicewiseAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='slicewise_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_slices = self.add_weight(name='ts', initializer='zeros')
        self.correct_slices = self.add_weight(name='cs', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)
        total = tf.cast(tf.reduce_sum(y_true), tf.float32)  # one-hot
        y_valid = tf.cast(tf.reduce_sum(y_true, axis=2), tf.bool)
        
        y_true = tf.argmax(y_true, axis=2)
        y_pred = tf.argmax(y_pred, axis=2)
        correct = (y_true == y_pred) & y_valid
        correct = tf.reduce_sum(tf.cast(correct, tf.float32))
        
        self.total_slices.assign_add(total)
        self.correct_slices.assign_add(correct)

    def result(self):
        return self.correct_slices / self.total_slices

    def reset_states(self):
        self.total_slices.assign(0)
        self.correct_slices.assign(0)

In [ ]:
acc = SlicewiseAccuracy()
acc.update_state(np.array([[[1, 0], [0, 0]]]), np.array([[[1, 0], [0, 1]]]))

In [ ]:
acc.result()

In [ ]:
data = Data()

In [ ]:
phase_gen = PhaseDataGenerator(data, "KAG", target_size=(224, 224), batch_size=2, shuffle=True)

In [ ]:
slice_gen = SliceDataGenerator(data, "KAG", target_size=(224, 224), batch_size=32, shuffle=True)

In [ ]:
assert(np.isclose(slice_gen[0][0][0].min(), -1, rtol=1.0e-4))
assert(np.isclose(slice_gen[0][0][0].max(), 1, rtol=1.0e-4))

In [ ]:
assert(np.isclose(phase_gen[0][0][0].min(), -1, rtol=1.0e-4))
assert(np.isclose(phase_gen[0][0][0].max(), 1, rtol=1.0e-4))

In [ ]:
keras_app = tf.keras.applications.mobilenet
keras_model = tf.keras.applications.mobilenet.MobileNet
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1,
    fill_mode="nearest",
    preprocessing_function=keras_app.preprocess_input)

In [ ]:
backbone = keras_model(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3))
backbone.trainable = False

In [ ]:
cnn_model = Sequential()
cnn_model.add(backbone)
cnn_model.add(Dense(256, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(5, activation="softmax"))

cnn_model.layers[0].trainable = False
cnn_model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [ ]:
cnn_model.fit(slice_gen, epochs=20)

In [ ]:
rnn_model = Sequential()
rnn_model.add(TimeDistributed(backbone))
rnn_model.add(LSTM(256, input_shape=(25, 2048), return_sequences=True))
rnn_model.add(Dropout(0.5))
#rnn_model.add(TimeDistributed(Dense(256, activation="relu")))
#rnn_model.add(TimeDistributed(Dropout(0.5)))
rnn_model.add(TimeDistributed(Dense(5, activation="softmax")))

In [ ]:
rnn_model.layers[0].trainable = False
rnn_model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=[SlicewiseAccuracy(), "accuracy"])

In [ ]:
rnn_model.fit(phase_gen, epochs=20)

In [ ]:
test_gen = PhaseDataGenerator(data, "KAG", target_size=(224, 224), batch_size=2, shuffle=False)

In [ ]:
rnn_model.evaluate(test_gen)

In [ ]:
preds = rnn_model.predict(test_gen)

In [ ]:
labels = None
for _, batch in test_gen:
    if labels is None:
        labels = batch
    else:
        labels = np.concatenate([labels, batch])
    print(labels.shape)

In [ ]:
preds.shape

In [ ]:
def get_accuracy(preds, labels):
    total_slices = 0
    total_phases = 0
    correct_slices = 0
    correct_phases = 0

    for phase, slices in enumerate(preds):
        total_phases += 1
        correct_phase = True
        for slice, pred in enumerate(slices):
            label = labels[phase][slice]
            if label.sum() == 0:
                pass
            else:
                total_slices += 1
                if label.argmax() == pred.argmax():
                    correct_slices += 1
                else:
                    correct_phase = False
        if correct_phase:
            correct_phases += 1

    slice_accuracy = correct_slices / total_slices
    phase_accuracy = correct_phases / total_phases

    return slice_accuracy, phase_accuracy, total_slices

In [ ]:
slice_accuracy, phase_accuracy, ts = get_accuracy(preds, labels)
print("Slice accuracy:", slice_accuracy)
print("Phase accuracy:", phase_accuracy)

In [ ]:
print(ts)

In [ ]:
ts / (preds.shape[0] * 25)